# Exercise 3: Model Predictive Control

> Welcome to the Advanced Robot Learning and Decision Making exercises!

In this exercise you will learn how to develop your own MPC algorithms to stabilize the drone to a certain position. 

1. **Recap of MPC concepts**

2. **Introduction to Acados**

3. **Linear MPC**

4. **Non-linear MPC**

5. **Compare linear MPC, non-linear MPC, and PD**

6. **Investigate influence of noisy observations**


_________


## 1 MPC Essentials

In this chapter, we will introduce model predictive control (MPC) for solving discrete-time nonlinear optimal control problems subject to state and input constraints. Constraints  reflect limits in states and actuation in the real world. This is important as a controller that is unaware of state and input constraints can yield an unstable closed-loop system or can result in a system failure through state constraint violation.

We will introduce conditions such that the MPC approximates an infinite-horizon optimal control problem by solving a finite-horizon optimal control problem at every time step. 

Unless otherwise mentioned we are considering the case of stabilizing a system to an equilibrium at the origin.

### 1.1 Notation
In the following, $ x_k $ is the state of the system at time step $ k $. The state $ x_{k + i \vert k} $ is the state of the system at time step $ k + i $, predicted at time step $ k $ by starting from $ x_{k \vert k} = x_k $ and applying the sequence of control inputs $ u_{k \vert k}, \dots, u_{k + i - 1 \vert k} $ to the system model:

$$
x_{k + 1 \vert k} = f(x_{k \vert k}, u_{k \vert k})
$$

Analogously, $ u_{k + i \vert k} $ is the control input at time $ k + i $ computed at time $ k $.

### 1.2 Model Predictive Control Formulation
Since an infinite-horizon-constrained optimal control problem has infinitely many variables, solving it directly is generally intractable. Instead of solving the infinite-horizon problem, we consider a truncated version. The goal of **Model Predictive Control (MPC)** is to find the optimal control input sequence over a finite horizon of $ N > 0 $ steps at every time step.

At every time step $ k $, the MPC solves the following finite-horizon constrained optimal control problem:

$$
J_k^*(x_k) = \underset{u_{k \vert k}, \dots, u_{k + N - 1 \vert k} }{\text{min}} 
\quad \underbrace{g_N(x_{k + N \vert k})}_{\text{Terminal cost}} 
+ \sum_{i = 0}^{N - 1} g(x_{k + i \vert k}, u_{k + i \vert k})
$$

subject to:

$$
\begin{aligned}
    & x_{k + i + 1 \vert k} = f(x_{k + i \vert k}, u_{k + i \vert k}), \quad \forall i \in \{0, \dots, N - 1\} \,, \\
    & x_{k + i \vert k} \in \mathcal{S}, \quad u_{k + i \vert k} \in \mathcal{U} \,, \quad \forall i \in \{0, \dots, N - 1\} \,, \\
    & \underbrace{x_{k + N \vert k} \in \mathcal{S}_{\mathrm{f}}}_{\text{Terminal constraint}}, \\
    & x_{k \vert k} = x_k \,.
\end{aligned}
$$

where:
- $ g_N(x_{k+N|k}) $ represents the **terminal cost**.
- $ \mathcal{S}_{\mathrm{f}} $ is the **terminal constraint set**, which is relevant for stability and feasibility of the MPC.
- The finite-horizon length is **$ N $**, which defines how far the controller looks ahead.

The optimization yields the optimal input sequence:

$$
\{ u^*_{k \vert k}, \dots, u^*_{k + N - 1 \vert k} \}
$$

The first control input of this sequence is then applied to the system:

$$
u_k = u^*_{k \vert k}
$$

At the next time step, the optimization problem is **re-solved** at state $ x_{k + 1} $ with horizon $ N $. This iterative process is called **receding horizon control**, where the optimization problem is solved at every step with a shifting horizon.

## 2 Introduction to Acados

[**Acados**](https://docs.acados.org) is an open-source, high-performance optimization framework specifically designed for **Model Predictive Control (MPC)** and **nonlinear optimal control** problems. Developed with embedded systems in mind, it combines cutting-edge numerical optimization with practical deployment capabilities.

### Key Features:
- 🚀 **Embedded-Ready**: Generates self-contained C code for deployment on resource-constrained hardware (ARM Cortex-M, Raspberry Pi, etc.)
- ⚡ **Real-Time Performance**: Implements state-of-the-art algorithms (Sequential Quadratic Programming/SQP, Interior Point Methods) with linear algebra optimizations
- 🐍 **Python Integration**: Full-featured Python interface (`acados_template`) for rapid prototyping in Jupyter environments
- 🔧 **Nonlinear MPC Support**: Handles complex dynamics through CasADi-based symbolic modeling
- 📦 **Modular Architecture**: Separates solver formulation, numerical routines, and system interfaces

```python
# Minimal Python Example
from acados_template import AcadosOcp, AcadosOcpSolver
ocp = AcadosOcp()
ocp.model = ...  # Define your dynamics using CasADi
ocp_solver = AcadosOcpSolver(ocp)  # Generates optimized C code

### 2.1 `class` AcadosOcp

[**AcadosOcp**](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp.AcadosOcp) is the primary Python object used to define and solve an optimal control problem (OCP) in the **acados** framework. It allows you to specify the system dynamics, constraints, and cost function, and configure solver options. Once the problem is defined, it generates a solver([`AcadosOcpSolver`](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_solver.AcadosOcpSolver)) that computes the optimal control inputs for your model.

### Key Properties

1. **model ([`AcadosModel`](https://docs.acados.org/python_interface/index.html#acados_template.acados_model.AcadosModel))**  
   Describes the system’s dynamics, including the definition of state and control variables, as well as any functions required (e.g., state update equations and corresponding Jacobians).

2. **constraints ([`AcadosConstraints`](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_constraints.AcadosOcpConstraints))**  
   Specifies bounds on states and controls, as well as any equality or inequality constraints and initial conditions.

3. **cost ([`AcadosCost`](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_cost.AcadosOcpCost))**  
   Defines the objective function to be minimized. This typically involves the stage costs and terminal cost, with matrices penalizing deviations from desired states or control inputs.

4. **solver_options ([`AcadosOcpOptions`](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_options.AcadosOcpOptions))**  
   Configures the numerical solver, including integration method, QP solver, discretization scheme, tolerance, and maximum iterations.

5. **code_export_directory (`str`)**  
   Determines where **acados** will place the generated C code when exporting the solver. This is especially useful for integrating the solver into embedded systems or other deployment environments.

By customizing these properties, **AcadosOcp** can be adapted to a wide variety of Model Predictive Control (MPC) scenarios, ranging from simple academic examples to complex real-world applications.

### 2.2 `class` AcadosModel

[**AcadosModel**](https://docs.acados.org/python_interface/index.html#acados_template.acados_model.AcadosModel) is used to define the system model within the **acados** framework. It includes details about the system’s states, inputs, and the expressions defining the system dynamics. Below are the key properties:

1. **`x`**  
   `CasADi` variable represents the state vector of the system. Each element corresponds to a particular state variable.

2. **`u`**  
   `CasADi` variable denotes the control input vector. Each element corresponds to a control variable that can be manipulated.

3. **`xdot`**  
   `CasADi` variable specifies the time derivative of the state vector. This is especially useful in implicit formulations of system dynamics.

4. **`f_expl_expr`**  
   `CasADi` expression for the explicit dynamics expression, typically used when the system is defined in the form  
   $$
   \dot{x} = f(x, u).
   $$
   Used if `AcadosOcpOptions.integrator_type` == 'ERK'.

5. **`f_impl_expr`**  
   `CasADi` expression for the implicit dynamics expression, used for systems described in the form  
   $$
   f(\dot{x}, x, u) = 0.
   $$
   Used if `AcadosOcpOptions.integrator_type` == 'IRK'.

6. **`cost_y_expr`**  
   The `CasADi` expression used in acados to define the nonlinear output of your system that appears in the least-squares cost function.
   When using the `NONLINEAR_LS` (Nonlinear Least Squares) cost type in acados, `cost_y_expr` specifies how to:

   - Compute the difference between predicted outputs and references ($y - y_{ref}$)
   - Define custom residuals for trajectory optimization or tracking tasks

   **Mathematical Form**:  
   $$
   \text{Cost} = \| y - y_{\text{ref}} \|^2_W
   $$
   - $ y = \text{cost\_y\_expr} $: Output defined by your expression.  
   - $ W $: Weighting matrix (set via `cost_W` in `3.2.4`).  

7. **`cost_y_expr_e`**  
   The `CasADi` expression used to define the terminal cost component \( y_e \), typically involving the final state at the end of the prediction horizon.

By customizing these properties, **AcadosModel** allows you to represent a wide range of system behaviors, making it straightforward to set up dynamic models for Model Predictive Control applications.

### 2.3 `class` AcadosConstraints

[**AcadosConstraints**](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_constraints.AcadosOcpConstraints) is responsible for defining the constraints of the optimal control problem (OCP) within the **acados** framework, including bounds on states and controls, and general linear constraints. Below are the key properties:

- **`x0`**  
  Initial state of the system.

- **`lbx` / `ubx`**  
  Lower and upper bounds on the state variables over the prediction horizon.

- **`idxbx`**  
  Indices specifying which state variables are bounded by `lbx` and `ubx`.

- **`lbx_e` / `ubx_e`**  
  Lower and upper bounds on the state variables at the terminal time.

- **`idxbx_e`**  
  Indices specifying which state variables are bounded at the terminal time.

- **`lbu` / `ubu`**  
  Lower and upper bound on the control inputs.

- **`idxbu`**  
  Indices specifying which control inputs are bounded.

- **`C`, `D`, `lg`, `ug`**  
  Matrices and vectors describing general linear constraints of the form  
  $$
    lg \leq Cx + Du \leq ug.
  $$
`Acados` supports multiple types of constraints, including:
  1. **Bounding box constraints**: State and input bounds (`lbx`, `ubx`, `lbu`, `ubu`).
  2. **Linear constraints**: `lg <= Cx + Du <= ug`.
  3. **Nonlinear constraints**: `lh <= h(x, u) <= uh`.


### 2.4 `class` AcadosCost

[**AcadosCost**](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_cost.AcadosOcpCost) defines the cost function of the OCP, determining how states and controls are penalized. The key properties include:

- **`cost_type`**  
  Specifies the type of the stage cost (e.g., `LINEAR_LS`, `NONLINEAR_LS`).

- **`cost_type_e`**  
  Specifies the type of the terminal cost (e.g., `LINEAR_LS`, `NONLINEAR_LS`).

- **`W`**  
  Weight matrix for the stage cost, penalizing deviations in states `and` controls.

- **`W_e`**  
  Weight matrix for the terminal cost, penalizing deviations in the final state.

- **`yref`**  
  Reference vector used in the cost function for stage costs. For instance, this might be a vector combining certain state `and` control components.

- **`yref_e`**  
  Reference vector for the terminal state, used in the terminal cost formulation.

- **`Vx, Vu, Vx_e`**

  If `cost_type` is `LINEAR_LS`, the output \( y(x, u) \) is typically defined as a **linear mapping** of the state \( x \)  and control \( u \):
  $$
  \begin{aligned}
  y(x, u) &= V_x x + V_u u \\
  y_e(x) &= V_{x\_e} x
  \end{aligned}
  $$



### 2.5 `class` AcadosOcpOptions

[**AcadosOcpOptions**](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_options.AcadosOcpOptions) collects various solver settings and algorithmic parameters for the OCP. Below are its key properties:

- **`N_horizon`**  
  The prediction horizon. (number of time step)

- **`tf`**  
  The prediction horizon in seconds.

- **`nlp_solver_type`**  
  The type of NLP (Nonlinear Programming) solver.
  This option determines the high-level algorithm used to solve the nonlinear programming (NLP) problem.  
  Common values include:
	-	`SQP`: **Sequential Quadratic Programming (SQP)**, which performs multiple iterations at each time step until convergence is reached.
	-	`SQP_RTI`: **Real-Time Iteration (RTI) SQP**, which performs only one SQP iteration per time step, sacrificing some accuracy for faster computation.  

	In short, nlp_solver_type controls the **overall optimization framework** used to solve the NLP.

- **`hessian_approx`**  
  The type of Hessian approximation (e.g., `GAUSS_NEWTON`, `EXACT`). This option determines **how the Hessian (second-order derivative information) is approximated** in the numerical integration and sensitivity computations.  
  Common values include:
	-	`GAUSS_NEWTON`: Uses the **Gauss-Newton approximation**, ignoring second-order terms from the system dynamics. This is computationally efficient and widely used in practical control problems.
	-	`EXACT` or `EXACT_HESSIAN`: Computes the **exact Hessian**, including second-order terms from the system dynamics, leading to higher accuracy but increased computational cost.
	
  Thus, hessian_approx_integrator_type controls **whether the solver uses an exact Hessian or an approximation** when handling dynamics and cost function derivatives.

- **`integrator_type`**  
  This option determines **how the continuous-time dynamics are discretized** into a form suitable for the optimal control problem solver. Here are the main choices:
  - `ERK (Explicit Runge-Kutta)`
  - `IRK (Implicit Runge-Kutta)`
  - `GNSF (Generalized Nonlinear Static Feedback)`
  -  etc.

- **`qp_solver`**  
  This option specifies the **Quadratic Programming (QP) solver** used inside the SQP iterations.
	
  In acados, several QP solvers are available with different formulations, such as:
	-	`FULL_CONDENSING_HPIPM`, `PARTIAL_CONDENSING_HPIPM`, `FULL_CONDENSING_QPOASES`, etc.
	The key differences are:
	-	Full condensing transforms the entire multi-step OCP into a **single large QP**.
	-	Partial condensing retains more of the original structure, reducing the problem size while maintaining sparsity.
	
  **Different numerical solvers** (e.g., `HPIPM`, `qpOASES`, `OSQP`) have different numerical stability and performance characteristics.
	
  Thus, qp_solver determines **how the inner QP subproblems in SQP are solved**.

- **`nlp_solver_max_iter`**（Typical Values: 10 - 1000）  
  **Maximum number of iterations** for the NLP solver. If the solver reaches this number of iterations before satisfying the stopping criteria, it will terminate and return the best solution found so far. A higher value allows more iterations for convergence but increases computation time.

- **`nlp_solver_tol_comp`** (Typical Values: 1e-6 - 1e-8)  
  Tolerance for **complementarity conditions** in interior-point methods. Ensures that the **KKT complementarity conditions** are satisfied for constrained problems. If too high, the solver might return suboptimal solutions.

- **`nlp_solver_tol_eq`** (Typical Values: 1e-6 - 1e-8, lower means stricter)  
  Tolerance for **equality constraints**. Determines how accurately the solver needs to satisfy **equality constraints** (e.g., system dynamics, hard constraints). A **lower value** means equality constraints must be satisfied more precisely.

- **`nlp_solver_tol_stat`** (Typical Values: 1e-6 - 1e-8, lower means stricter)  
  Tolerance for the **stationarity condition** (first-order optimality). Controls how small the gradient of the **Lagrangian function** needs to be for the solver to consider the solution as optimal. A **lower value** enforces a stricter optimality condition, leading to higher accuracy but possibly requiring more iterations.

- **`nlp_solver_tol_ineq`** (Typical Values: 1e-6 - 1e-8, lower means stricter)  
  Tolerance for **inequality constraints**. Controls the tolerance for satisfying **inequality constraints** (e.g., input bounds, state limits). A **lower value** ensures tighter constraint satisfaction but may make the problem harder to solve.

- **`globalization_fixed_step_length`** (Typical Values: 0.1 - 1.0, with 1.0 being a full step)  
  This parameter defines the **fixed step size** used in the globalization strategy for the SQP method. It controls how much the step size is scaled during the line search process. If **too small**, convergence can be slow; if **too large**, the solver may diverge.

By configuring these options, you can fine-tune solver behavior to achieve the desired balance between accuracy, computational speed, and robustness for your Model Predictive Control application.

### 2.6 `class` AcadosOcpSolver

[AcadosOcpSolver](https://docs.acados.org/python_interface/index.html#acados_template.acados_ocp_solver.AcadosOcpSolver) is the central piece that **numerically solves** the nonlinear optimal control problem (OCP) described by an `AcadosOcp` object. Once you set up your **dynamics, constraints, cost function, and solver settings** in an `AcadosOcp`, you can create an `AcadosOcpSolver` instance and use it to:   
- **Initialize** the solver memory (e.g., with warm-start guesses).
- **Iterate** towards the optimal solution (`solve()`).
- **Extract** results (optimal states, inputs, cost, etc.).

Once you have your `AcadosOcp` ready, you can create the solver:

```python
from acados_template import AcadosOcp, AcadosOcpSolver

# Suppose 'ocp' is already configured
ocp_solver = AcadosOcpSolver(ocp, json_file="acados_ocp.json")
# json_file (optional): Saves the problem configuration in a JSON file for reference or debugging.
```

Key Methods:

- **`AcadosOcpSolver.set(stage: int, field: str, value: ndarray)`**  
  Updates the solver’s internal data for a specific time stage.
    - Examples:
        ```python
        ocp_solver.set(stage_idx, "x", x_init)
        ocp_solver.set(stage_idx, "u", u_warmstart)
        ocp_solver.set(stage_idx, "yref", y_reference)
        ocp_solver.set(stage_idx, "lbx", x_lower_bound)
        ...
        ```
    - If your references change at each sampling instant, you call set(i, "yref", ...) for each stage i.

- **`AcadosOcpSolver.solve()`**  
  Runs the numerical optimization (usually an SQP or SQP-RTI method) to find the optimal control sequence.
    - `Return Value`: An integer status code:
        - 0 → Converged successfully.
        - Nonzero → Encountered a convergence issue (e.g., max iterations reached, infeasibility, etc.).

- **`AcadosOcpSolver.get(stage: int, field: str)`**  
  Retrieves the result of the optimization or internal solver data from the specified stage.
    - Examples:
        ```python
        u_opt = ocp_solver.get(0, "u")
        x_opt_k = ocp_solver.get(k, "x")

**Hint:**
After a successful `solve()`, `get(0, "u")` typically provides the control action to apply **right now** in an MPC loop.

## 3 Implementing simple linear-MPC with Acados

In this section, we begin by implementing a simple linear MPC using a linear time-invariant (LTI) model. That is, the system dynamics are assumed to be linear and do not change over time.

In [ ]:
%reload_ext autoreload
%autoreload 2

import time

import gymnasium
import matplotlib.pyplot as plt
import numpy as np
import yaml
from crazyflow.constants import GRAVITY, MASS
from linear_mpc import LinearModelPredictiveController
from noisy_obs_wrapper import NoisyObservationWrapper
from nonlinear_mpc import NonlinearModelPredictiveController
from utils import obs_to_state

SEED = 42

In [ ]:
env = gymnasium.make_vec(
    "DroneReachPos-v0",
    num_envs=1,
    freq=500,
    device="cpu",

)

print('observation space: \n', env.observation_space)
print('action space: \n', env.action_space)
print('time step:', 1 / env.sim.freq)

<div class="alert alert-warning">
    <h3>Note: Action Space</h3>
    <p>The action space is a continuous space defined by a Box object with a shape of <b>(1, 4)</b>. The range for the first variable is <b>0.11264675 to 0.5933658</b>, while the other three variables range from <b>-π/2 to π/2</b>. The actions are represented as <b>float32</b>.
         </p>
</div>


Implement the function `create_linear_prediction_model(symbolic_model: SymbolicModel) -> AcadosModel`. This function should define the `AcadosModel` object with the help of an existing symbolic model:
  - Define `model.x, model.u`
  - Assign `model.disc_dyn_expr`

<div class="alert alert-info">
    <h3>Task 1: Create Acados Model</h3>
    <p>
      Implement the function <code>create_linear_prediction_model(symbolic_model: SymbolicModel) -> AcadosModel</code> in <code>exercise03/mpc_utils.py</code>. This function defines the prediction model for a drone system.
    </p>
</div>



Next, complete the function `create_ocp_constraints(ocp: AcadosOcp, options: dict) -> AcadosOcp`. This function sets up **constraints** for the drone. Your task is to fill in the missing parts of the function by:

- **Fixing the initial state** `ocp.constraints.x0`
- **Defining input bounds**:
  - `ocp.constraints.lbu`: Lower bound on control inputs
  - `ocp.constraints.ubu`: Upper bound on control inputs
  - `ocp.constraints.idxbu`: Indices of constrained control inputs

<div class="alert alert-info">
    <h3>Task 2: Create OCP Constraints</h3>
    <p>
      Please implement the function <code>create_ocp_constraints(ocp: AcadosOcp, options: dict) -> AcadosOcp</code> in <code>exercise03/mpc_utils.py</code>. This function sets up constraints for a drone system.
    </p>
</div>

Next, you need to complete the function `create_ocp_costs_linear(ocp: AcadosOcp, options: dict) -> AcadosOcp`. This function defines the **cost function** for the drone. Your task is to fill in the missing parts of the function by:

- **Defining cost function settings**:
  - Set cost type of stage and terminal cost to `"LINEAR_LS"`
  - Assign stage cost weight matrix (`W`)
  - Assign terminal cost weight matrix (`W_e`)
  - Construct `Vx`, `Vu`, and `Vx_e` matrices to map state and input to the cost function.    
  - Initialize reference trajectories to zero (intermediate and terminal)

- **Hints**
  - `options` is a dictionary containing cost configuration parameters with 4 keys:
    - `cost_type`: `'LINEAR_LS'`
    - `cost_type_e`: `'LINEAR_LS'`
    - `Q`: List of diagonal elements for the **state weight matrix**
    - `R`: List of diagonal elements for the **input weight matrix**

<div class="alert alert-info">
    <h3>Task 3: Create OCP Cost Function</h3>
    <p>
      Please implement the function <code>create_ocp_costs_linear(ocp: AcadosOcp, options: dict) -> AcadosOcp</code> in <code>exercise03/mpc_utils.py</code>. This function defines the cost function for a drone system.
    </p>
</div>


Next, complete the function `create_ocp_solver(ocp: AcadosOcp, options: dict) -> AcadosOcp`. This function configures the **solver settings**. Your task is to fill in the missing parts of the function by:

- **Defining the prediction horizon**: Both in `time step` and in `second`.
  
- **Selecting the solver type**:
  - Set the type of the `NLP solver` and the `QP solver` to the corresponding values from the `options` dictionary.

- **Choosing numerical approximation methods**:
  - Set the type of the `hessian approximation` and the `integrator` to the corresponding values from the `options` dictionary.

- **Setting NLP solver parameters**:
  - Set `nlp_solver_max_iter`, `lp_solver_tol_stat`, `nlp_solver_tol_eq`, `lp_solver_tol_ineq`, `nlp_solver_tol_comp`, `nlp_solver_step_length` to default value in `options` dictionary.

- **Specifying the export directory**: Use the `code_export_directory` specified in the `configs/*_mpc_config.yaml`
  - `ocp.code_export_directory = "acados_c_code/MPC/c_generated_code"`: Directory where the generated C code is stored.

**Hints**
- `options` is a dictionary containing solver configuration parameters with 13 keys:
  - `export_dir`: path where acados will place the generated `C` code when exporting the solver  
  - `n_pred`: prediction horizon (time step)  
  - `Ts`: sampling time [s]  
  - `integrator_type`: type of Integrator  
  - `nlp_solver_type`: type of NLP solver  
  - `qp_solver`: type of QP solver  
  - `hessian_approx`: type of hessian approximation  
  - `nlp_solver_max_iter`: maximum number of iterations for the NLP solver.  
  - `nlp_solver_tol_comp`: tolerance for complementarity conditions in interior-point methods.  
  - `nlp_solver_tol_eq`: tolerance for equality constraints  
  - `nlp_solver_tol_stat`: tolerance for the stationarity condition  
  - `nlp_solver_tol_ineq`: tolerance for inequality constraints  
  - `nlp_solver_step_length`: step size in the SQP method  
- Multiply the number of time steps by the sampling time per step to obtain the time horizon in seconds.
<div class="alert alert-info">
    <h3>Task 4: Configure the OCP Solver Options</h3>
    <p>
      Please implement the function <code>create_ocp_solver(ocp: AcadosOcp, symbolic_model: SymbolicModel, options: dict) -> AcadosOcp</code> in <code>exercise_03/mpc_utils.py</code>. This function configures the solver settings for a drone system.
    </p>
</div>

<div class="alert alert-warning">
    <h3>Note: Config File</h3>
    <p>
    Open the <code>exercise03/configs/linear_mpc_config.yaml</code> file to see the default settings for various parameters in your linear-MPC. First keep the default parameters to finish the exercises and then feel free to tweak these parameters later to experiment and optimize your controller.
    </p>
</div>

In [ ]:
with open('configs/linear_mpc_config.yaml') as file:
    config_linear = yaml.load(file, Loader=yaml.FullLoader)
print(config_linear)

Use the four functions implemented above to complete the function `create_ocp_linear(symbolic_model: SymbolicModel, options: dict)`. This function constructs an **Acados OCP** step by step by:
1. Defining the OCP model
   
2. Adding constraints to the OCP
   
3. Defining the OCP cost function
   
4. Configuring the OCP solver

<div class="alert alert-info">
    <h3>Task 5: Construct the Complete Acados OCP</h3>
    <p>
      Please implement the function <code>create_ocp_linear(symbolic_model: SymbolicModel, options: dict)</code> in <code>exercise03/ocp_setup.py</code>. This function constructs an Acados OCP step by step.
    </p>
</div>


In [ ]:
linear_mpc = LinearModelPredictiveController(env, config_linear)

Finally, you need to compute the control input.
1. **Set initial state constraints**  
   Configure your OCP solver so that the lower bound (`lbx`) and upper bound (`ubx`) at time step 0 match the current state `x`.
   
2. **Set the reference trajectory**  
   Populate the solver’s reference (via `yref`) for all time steps in the horizon.

3. **Warm start** (Optional)    
   Optionally use the previous solution as a starting point for the new solve, which can speed up convergence.

4. **Solve the OCP**  
   Call the solver and check the return status. Handle failures gracefully (e.g., print a message or use a fallback control).

5. **Extract the control**  
   Retrieve the **first** computed control action from the solver.

6. **Store predictions**  
   Update any internal predicted trajectories for debugging or visualization (i.e., predicted states/controls).

**Hint**  
- Please use the methods of AcadosOcpSolver
- If the solver fails to converge, consider returning a safe fallback control (such as a zero vector or the previous control) to ensure system stability and safety.   
- Don't forget to reshape and convert the control input to the appropriate shape and dtype for your environment.

<div class="alert alert-info">
    <h3>Task 6: Compute Control Input</h3>
    <p>
      Please implement the function 
      <code>step_control(self, x: np.ndarray, y_ref: np.ndarray, y_ref_e: np.ndarray)</code>
      in
      <code>exercise_03/linear_mpc.py</code>. This function should compute the control input for the system 
      at each simulation step, update the solver’s internal states, and return the control action 
      that will be applied to the system.
    </p>
</div>


Read and run the following cell to regulate the drone to the given goal position. First keep the default goal point and then feel free to change it.

In [ ]:
obs, info = env.reset(seed=SEED)
state = obs_to_state(obs)

x_log_linear = [state[0]]
y_log_linear = [state[1]]
z_log_linear = [state[2]]
control_input_log_linear = []
fps = 60

# Define the equilibrium state and control input
# TODO experiment with different goal states
goal = np.array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

env.unwrapped._goal = env.unwrapped._goal.at[...].set(np.array([goal[0], goal[1], goal[2]])) # update goal in the environment for vizualization

goal_u = np.array([MASS*GRAVITY, 0, 0, 0])
y_ref = np.concatenate([goal, goal_u])
y_ref = np.tile(y_ref, (linear_mpc.N, 1))
y_ref_e = goal

for i in range(2500):
    action = linear_mpc.step_control(state, y_ref, y_ref_e)
    
    control_input_log_linear.append(action.flatten())
    obs, reward, terminated, truncated, info = env.step(action)

    state = obs_to_state(obs)
    x_log_linear.append(state[0])
    y_log_linear.append(state[1])
    z_log_linear.append(state[2])
    
    if terminated or truncated:
        print("Episode ended at step:", i)
        break

    if (i * fps) % env.sim.freq < fps:
        env.render()
        env.unwrapped.sim.viewer.viewer.cam.lookat = env.unwrapped._goal[0]
        time.sleep(1 / fps)
    
env.close()

## 4 Implementing nonlinear-MPC with Acados

In this task, we move on to implementing a nonlinear model predictive control (NMPC) scheme.

Again, you need to complete the function `create_nonlinear_prediction_model(symbolic_model: SymbolicModel) -> AcadosModel`. This function should define the `AcadosModel` object with the help of an existing symbolic model:
  - Define `model.x, model.u, model.xdot`
  - Assign `model.f_expl_expr, model.f_impl_expr`
  - Assign `model.cost_y_expr, model.cost_y_expr_e`

<div class="alert alert-info">
    <h3>Task 7: Create Acados Model</h3>
    <p>
      Please implement the function <code>create_nonlinear_prediction_model(symbolic_model: SymbolicModel) -> AcadosModel</code> in <code>exercise03/mpc_utils.py</code>. This function defines the prediction model for a drone system.
    </p>
</div>

Next, again complete the function `create_ocp_costs_nonlinear(ocp: AcadosOcp, options: dict) -> AcadosOcp`. This function defines the **cost function**. Your task is to fill in the missing parts of the function by defining the cost function settings:
  - Set cost type of stage and terminal cost to `"NONLINEAR_LS"`
  - Assign stage cost weight matrix (`W`)
  - Assign terminal cost weight matrix (`W_e`)
  - Initialize reference trajectories to zero (intermediate and terminal)

 **Hints**
- `options` is a dictionary containing cost configuration parameters with 4 keys:
   - `cost_type`: `'NONLINEAR_LS'`
   - `cost_type_e`: `'NONLINEAR_LS'`
   - `Q`: List of diagonal elements for the **state weight matrix**
   - `R`: List of diagonal elements for the **input weight matrix**

<div class="alert alert-info">
    <h3>Task 8: Create OCP Cost Function</h3>
    <p>
      Please implement the function <code>create_ocp_costs_nonlinear(ocp: AcadosOcp, options: dict) -> AcadosOcp</code> in <code>exercise03/mpc_utils.py</code>. This function defines the cost function for a drone system.
    </p>
</div>



<div class="alert alert-warning">
    <h3>Note: Config File</h3>
    <p>
    Open the <code>exercise03/nonlinear_mpc_config.yaml</code> file to see the default settings for various parameters in your nonlinear-MPC. First keep the default parameters to finish the exercises and then feel free to tweak these parameters later to experiment and optimize your controller.
    </p>
</div>

In [ ]:
with open('configs/nonlinear_mpc_config.yaml') as file:
    config_nonlinear = yaml.load(file, Loader=yaml.FullLoader)

print(config_nonlinear)

Use the four functions implemented above to complete the function `create_ocp_nonlinear(symbolic_model: SymbolicModel, options: dict)`. This function constructs an **Acados OCP** step by step by:
1. Defining the OCP model
   
2. Adding constraints to the OCP
   
3. Defining the OCP cost function
   
4. Configuring the OCP solver

<div class="alert alert-info">
    <h3>Task 9: Construct the Complete Acados OCP</h3>
    <p>
      Please implement the function <code>create_ocp_nonlinear(symbolic_model: SymbolicModel, options: dict)</code> in <code>exercise03/ocp_setup.py</code>. This function constructs an Acados OCP step by step.
    </p>
</div>


In [ ]:
nonlinear_mpc = NonlinearModelPredictiveController(env, config_nonlinear)

Finally, you need to compute the control input.

1. **Set initial state constraints**  
   Configure your OCP solver so that the lower bound (`lbx`) and upper bound (`ubx`) at time step 0 match the current state `x`.
   
2. **Set the reference trajectory**  
   Populate the solver’s reference (via `yref`) for all time steps in the horizon.

3. **Warm start** (Optional)    
   Optionally use the previous solution as a starting point for the new solve, which can speed up convergence.

4. **Solve the OCP**  
   Call the solver and check the return status. Handle failures gracefully (e.g., print a message or use a fallback control).

5. **Extract the control**  
   Retrieve the **first** computed control action from the solver.

6. **Store predictions**  
   Update any internal predicted trajectories for debugging or visualization (i.e., predicted states/controls).

 **Hints**  
 - Please use the methods of AcadosOcpSolver
 - If the solver fails to converge, consider returning a safe fallback control (such as a zero vector or the previous control) to ensure system stability and safety.   
 - Don't forget to reshape and convert the control input to the appropriate shape and dtype for your environment.

<div class="alert alert-info">
    <h3>Task 10: Compute Control Input</h3>
    <p>
      Please implement the function 
      <code>step_control(self, x: np.ndarray, y_ref: np.ndarray, y_ref_e: np.ndarray)</code>
      in
      <code>exercise_03/nonlinear_mpc.py</code>. This function should compute the control input for the system 
      at each simulation step, update the solver’s internal states, and return the control action 
      that will be applied to the system.
    </p>
</div>

Run the following cell to regulate the drone to the given goal position. First keep the default goal point and then feel free to change it.

In [ ]:
obs, info = env.reset(seed=SEED)
state = obs_to_state(obs)
# print(obs)
# Step through the environment
x_log = [state[0]]
y_log = [state[1]]
z_log = [state[2]]
control_input_log = []
fps = 60

# Define the equilibrium state and control input
# TODO experiment with different goal states
goal = np.array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

env.unwrapped._goal = env.unwrapped._goal.at[...].set(np.array([goal[0], goal[1], goal[2]])) # update goal in the environment for vizualization

goal_u = np.array([MASS*GRAVITY, 0, 0, 0])
y_ref = np.concatenate([goal, goal_u])
y_ref = np.tile(y_ref, (nonlinear_mpc.N, 1))
y_ref_e = goal

for i in range(2500):
    action = nonlinear_mpc.step_control(state, y_ref, y_ref_e)
    
    control_input_log.append(action.flatten())
    obs, reward, terminated, truncated, info = env.step(action)

    state = obs_to_state(obs)
    x_log.append(state[0])
    y_log.append(state[1])
    z_log.append(state[2])
    
    if terminated or truncated:
        print("Episode ended at step:", i)
        break
    if (i * fps) % env.sim.freq < fps:
        env.render()
        env.unwrapped.sim.viewer.viewer.cam.lookat = env.unwrapped._goal[0]
        time.sleep(1 / fps)
    
env.close()

For comparison, we also run the PD controller from exercise 01.

In [ ]:
from crazyflow.constants import GRAVITY, MASS
from PD import PDDrone

obs, info = env.reset(seed=SEED)
state = obs_to_state(obs)

x_log_pd = [state[0]]
y_log_pd = [state[1]]
z_log_pd = [state[2]]
control_input_log_pd = []

# PDController can only take xyz as target
goal_pd = np.array([goal[0], goal[1], goal[2]])

# PDController from exercise01
controller = PDDrone(
    des_pos=goal_pd,
    kp=np.array([0.4, 0.4, 1.25]),  # kp gain
    kd=np.array([0.2, 0.2, 0.5]),  # kd gain
    drone_mass=MASS,
)

for i in range(2500):
    pos, vel, quat = obs["pos"].squeeze(), obs["vel"].squeeze(), obs["quat"].squeeze()
    action = controller.step_control(pos, vel, quat)
    action = np.clip(
        action, env.unwrapped.action_space.low, env.unwrapped.action_space.high
    )  # bound actions otherwise they are rejected by the step function
    obs, _, terminated, truncated, _ = env.step(action)

    state = obs_to_state(obs)
    x_log_pd.append(state[0])
    y_log_pd.append(state[1])
    z_log_pd.append(state[2])

    control_input_log_pd.append(action.flatten())

    if terminated or truncated:
        env.reset()
env.close()
env.unwrapped.sim.close()

print(f"Final position of the drone: {obs['pos']}")

### 4.1 Plot of positions

In [ ]:
# Generate time array based on fixed step interval
time_log_state = np.arange(len(x_log)) / env.sim.freq
time_log_state_linear = np.arange(len(x_log_linear)) / env.sim.freq

# Plot theta and control input vs. time
plt.figure(figsize=(10, 5))
plt.plot(time_log_state, x_log, label="x(non-linear)", color="blue")
plt.plot(time_log_state, y_log, label="y(non-linear)", color="green")
plt.plot(time_log_state, z_log, label="z(non-linear)", color="red")
plt.plot(time_log_state_linear, x_log_linear, label="x(linear)", color="blue", linestyle="--")
plt.plot(time_log_state_linear, y_log_linear, label="y(linear)", color="green", linestyle="--")
plt.plot(time_log_state_linear, z_log_linear, label="z(linear)", color="red", linestyle="--")
plt.plot(time_log_state_linear, x_log_pd, label="x(pd)", color="blue", linestyle="dotted")
plt.plot(time_log_state_linear, y_log_pd, label="y(pd)", color="green", linestyle="dotted")
plt.plot(time_log_state_linear, z_log_pd, label="z(pd)", color="red", linestyle="dotted")
plt.xlabel("Time (s)")
plt.ylabel("Position")
plt.title("Position vs Time")
plt.legend()
plt.grid()
plt.show()

### 4.2 Plots of control inputs

In [ ]:
# Convert logs to numpy arrays
control_input_log = np.array(control_input_log)
control_input_log_linear = np.array(control_input_log_linear)
control_input_log_pd = np.array(control_input_log_pd)

# Calculate time vectors
time_log_action = np.arange(control_input_log.shape[0]) / env.sim.freq

# Define control input labels and colors
labels = [
    "normalized thrust [N]",
    "desired roll angle [rad]",
    "desired pitch angle [rad]",
    "desired yaw angle [rad]",
]
colors = ["blue", "orange", "green", "red"]


def plot_control_inputs(time_data, control_data, title, linestyle="-"):
    plt.figure(figsize=(10, 5))

    for i in range(4):
        plt.plot(
            time_data,
            control_data[:, i],
            label=f"{labels[i]} ({title})",
            color=colors[i],
            linestyle=linestyle,
        )

    plt.xlabel("Time Steps")
    plt.ylabel("Control Input")
    plt.title(f"Control Input over Time ({title})")
    plt.legend()
    plt.grid()
    plt.show()


# Create the three plots
plot_control_inputs(time_log_action, control_input_log, "nonlinear-MPC")
plot_control_inputs(time_log_action, control_input_log_linear, "linear-MPC", linestyle="--")
plot_control_inputs(time_log_action, control_input_log_pd, "PD", linestyle="--")


## 5. Comparisons

<div class="alert alert-success">
    <h3>Task 11: Exam Preparation</h3>
    <p>
    Observe the simulation results and comment on the behaviour of the MPC controllers.
    </p>
    <p>
    Submission is not required, but it’s crucial for the exam.
    </p>
</div>

<div class="alert alert-success">
    <h3>Task 12: Exam Preparation</h3>
    <p>
    Analyze the results of the linear and nonlinear MPC implementations, and consider the respective strengths and limitations of each method. How does the PD control compare to MPC? What are the advantages and disadvantages of PD?
    </p>
    <p>
    Submission is not required, but it’s crucial for the exam.
    </p>
</div>

<div class="alert alert-success">
    <h3>Task 13: Exam Preparation</h3>
    <p>
    What effect do different lengths of the prediction horizon have? (What happens when the prediction horizon is 1?) Change the respective parameters in the config file and explain your observations.
    </p>
    <p>
    Submission is not required, but it’s crucial for the exam.
    </p>
</div>

## 6. Influence of observation noise

<div class="alert alert-success">
    <h3>Task 14: Exam Preparation</h3>
    <p>
    Run the following cells and test the MPC controller with noisy measurements. Is the drone still able to reach the goal? Feel free to change <code>noise_std</code>.
    </p>
    <p>
    Submission is not required, but it’s crucial for the exam.
    </p>
</div>

In [ ]:
obs, _ = env.reset(seed=SEED)
print("Observation(without Noise):", obs)

In [ ]:
env_with_noise = NoisyObservationWrapper(env, noise_std=0.05)
obs, _ = env_with_noise.reset(seed=SEED, options=None)
print("Noisy Observation:", obs)

In [ ]:
obs, info = env_with_noise.reset(seed=SEED)
state = obs_to_state(obs)
# print(obs)
# Step through the environment
x_log_noisy = [state[0]]
y_log_noisy = [state[1]]
z_log_noisy = [state[2]]
control_input_log_noisy = []
fps = 60

# Define the equilibrium state and control input
# TODO experiment with different goal states
goal = np.array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

goal_u = np.array([MASS*GRAVITY, 0, 0, 0])
y_ref = np.concatenate([goal, goal_u])
y_ref = np.tile(y_ref, (nonlinear_mpc.N, 1))
y_ref_e = goal

for i in range(2500):
    action = nonlinear_mpc.step_control(state, y_ref, y_ref_e)
    
    control_input_log_noisy.append(action.flatten())
    obs, reward, terminated, truncated, info = env_with_noise.step(action)

    state = obs_to_state(obs)
    x_log_noisy.append(state[0])
    y_log_noisy.append(state[1])
    z_log_noisy.append(state[2])
    
    if terminated or truncated:
        print("Episode ended at step:", i)
        break

env_with_noise.close()

In [ ]:
time_log_state_noisy = np.arange(len(x_log_noisy)) / env.sim.freq

# Plot theta and control input vs. time
plt.figure(figsize=(10, 5))
plt.plot(time_log_state_noisy, x_log_noisy, label="x", color="blue")
plt.plot(time_log_state_noisy, y_log_noisy, label="y", color="green")
plt.plot(time_log_state_noisy, z_log_noisy, label="z", color="red")
plt.xlabel("Time (s)")
plt.ylabel("position")
plt.title("position vs Time (MPC with Noise)")
plt.legend()
plt.grid()
plt.show()

control_input_values_noisy = np.array(control_input_log_noisy)  # shape: (steps, 4)

time_log_state_noisy = np.arange(control_input_values_noisy.shape[0]) / env.sim.freq

plt.figure(figsize=(10, 5))
plt.plot(time_log_state_noisy, control_input_values_noisy[:, 0], label="normalized thrust [N] (MPC)", color="blue")
plt.plot(time_log_state_noisy, control_input_values_noisy[:, 1], label="desired roll angle [rad] (MPC)", color="orange")
plt.plot(time_log_state_noisy, control_input_values_noisy[:, 2], label="desired pitch angle [rad] (MPC)", color="green")
plt.plot(time_log_state_noisy, control_input_values_noisy[:, 3], label="desired yaw angle [rad] (MPC)", color="red")

plt.xlabel('Time Steps')
plt.ylabel('Control Input')
plt.title('Control Input over Time (MPC with Noise)')
plt.legend()
plt.grid()
plt.show()